# 1. Prepare Data
The 'extract_rgbnir_from_S2_folder' function creates a georeferenced .tiff of the RGB-NIR bands from the '.SAFE' data structure as provided by Scihub. The stacked image is saved in the same directory.

In [1]:
# if we have raw S2 download, create stacked image
from utils.rgbnir_stacked_from_S2_folder   import extract_rgbnir_from_S2_folder
from utils.bands20m_stacked_from_S2_folder import extract_20mbands_from_S2_folder

stack_needed=True
if stack_needed:
    folder_path = "/data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE"
    extract_rgbnir_from_S2_folder(folder_path)
    extract_20mbands_from_S2_folder(folder_path)

writing band R ...
writing band G ...
writing band B ...
writing band NIR ...
Image succesfully written to: /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/stacked_RGBNIR.tif
writing band RE1 ...
writing band RE2 ...
writing band RE3 ...
writing band NIR2 ...
writing band SWIR1 ...
writing band SWIR2 ...
Image succesfully written to: /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/stacked_20m.tif


# 2. SR functions
## 2.1 Create SR Object
This class holds all the information and function to SR a stacked RGB-NIR image. When initialized it holds the rasterio windowed reading coordinates with an overlap. The 'super_resolute_whole_image' function iteratively SRs the windows and cobines them together, averaging out the values for the overlapping areas.  
**Note some inefficiency problems:**
- Doesn't support multi-GPU inference
- The whole initialized SR tensor is stored in memory #ToDo: change to windowed writing with rasterio
- images are SR'd one by one, not batched
- other functionalities, like histogram matching, run single-threadedly

In [3]:
# create object that holds window coordinates etc
from main import windowed_SR_and_saving
file_path="/data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/stacked_RGBNIR.tif"
sr_obj = windowed_SR_and_saving(file_path)

In [8]:
!cp /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/SR_10mbands.tif /data1/simon/GitHub/sr-s2-tile

In [33]:
!ls -alh /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/

total 5.8G
drwxrwxrwx 7 simike visitor 4.0K Dec  8 17:35 .
drwxr-xr-x 3 simike visitor 4.0K Nov  2 10:14 ..
drwxrwxrwx 2 simike visitor 4.0K Jul 29 14:54 AUX_DATA
drwxrwxrwx 3 simike visitor 4.0K Jul 29 14:54 DATASTRIP
drwxrwxrwx 3 simike visitor 4.0K Jul 29 14:54 GRANULE
drwxrwxrwx 2 simike visitor 4.0K Jul 29 14:54 HTML
-rw-rw-rw- 1 simike visitor  19K Jul 29 14:54 INSPIRE.xml
-rw-rw-rw- 1 simike visitor  68K Jul 29 14:54 manifest.safe
-rw-rw-rw- 1 simike visitor  54K Jul 29 14:54 MTD_MSIL2A.xml
drwxrwxrwx 2 simike visitor 4.0K Jul 29 14:54 rep_info
-rw-r--r-- 1 simike visitor 5.4G Dec  8 17:40 SR_20mbands.tif
-rw-r--r-- 1 simike visitor 345M Dec  8 17:35 stacked_20m.tif


In [18]:
!ls -alh /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/SR_10mbands.tif

-rw-r--r-- 1 simike visitor 15G Dec  8 16:09 /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/SR_10mbands.tif


In [31]:
!rm /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/SR_20mbands.tif
!rm /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/stacked_20m.tif

In [13]:
!cp /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/stacked_RGBNIR.tif /data1/simon/GitHub/sr-s2-tile

In [19]:
!cp /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/SR_10mbands.tif /data1/simon/GitHub/sr-s2-tile

In [20]:
!cp /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/SR_20mbands.tif /data1/simon/GitHub/sr-s2-tile

In [28]:
!ls /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE

AUX_DATA   GRANULE  INSPIRE.xml    MTD_MSIL2A.xml
DATASTRIP  HTML     manifest.safe  rep_info


# 3. Load Model

In [7]:
!ls -alh /data2/simon/test_s2/S2A_MSIL2A_20230729T100031_N0509_R122_T33TUG_20230729T134559.SAFE/

total 60G
drwxrwxrwx 7 simike visitor 4.0K Dec  7 15:26 .
drwxr-xr-x 3 simike visitor 4.0K Nov  2 10:14 ..
drwxrwxrwx 2 simike visitor 4.0K Jul 29 14:54 AUX_DATA
drwxrwxrwx 3 simike visitor 4.0K Jul 29 14:54 DATASTRIP
drwxrwxrwx 3 simike visitor 4.0K Jul 29 14:54 GRANULE
drwxrwxrwx 2 simike visitor 4.0K Jul 29 14:54 HTML
-rw-rw-rw- 1 simike visitor  19K Jul 29 14:54 INSPIRE.xml
-rw-rw-rw- 1 simike visitor  68K Jul 29 14:54 manifest.safe
-rw-rw-rw- 1 simike visitor  54K Jul 29 14:54 MTD_MSIL2A.xml
drwxrwxrwx 2 simike visitor 4.0K Jul 29 14:54 rep_info
-rw-r--r-- 1 simike visitor  15G Dec  7 15:26 SR_10mbands.tif
-rw-r--r-- 1 simike visitor 5.4G Dec  7 15:26 SR_20mbands.tif
-rw-r--r-- 1 simike visitor 345M Dec  7 13:14 stacked_20m.tif
-rw-r--r-- 1 simike visitor  58G Nov  8 13:49 stacked_RGBNIR_SR.tif
-rw-r--r-- 1 simike visitor 1.1G Nov  8 14:13 stacked_RGBNIR_SR_window.tif
-rw-r--r-- 1 simike visitor 920M Dec  7 13:14 stacked_RGBNIR.tif


## 4. Perform SR
The 'super_resolute_whole_image' takes the initialized model and iteratively performs the SR and the stitching & saving of the final image. Our models typically dont have a standard *model.forward* call to perform SR, which is why you can specify a custom command. Note that the normalization parameters are set in the class definitions.

In [ ]:
# SR and save whole image
sr_obj.super_resolute_whole_image(model=None,forward_call="forward",custom_steps=100)